<a href="https://colab.research.google.com/github/JinFree/Thermo-and-Fluid-Engineering-Lab.1/blob/master/Thermo_and_Fluid_Engineering_Lab_1_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numba

## First Steps with numba

In [0]:
import numba
print(numba.__version__)

### Numba
  


* numba는 Python 코드의 특정 부분을 컴파일할 수 있게 하며, 그를 통해 해당 부분이 C 언어로 컴파일된 코드와 같은 속도로 작동할 수 있게 한다.
* numba는 함수 수순에서 작동한다.
* 이 노트북은 numba의 간단한 사용법이 설명되어 있으며, 아래 깃헙 저장소에서 소개된 내용을 한국어로 번역한 것이다.
  * https://github.com/numba/numba

* 간단한 예제
 * Numpy array를 활용하여 Python으로 버블소트를 구현한 경우

In [0]:
def bubblesort(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

* 우선 Numpy array를 생성한 후 랜덤하게 섞는다.

In [0]:
import numpy as np

original = np.arange(0.0, 10.0, 0.01, dtype='f4')
shuffled = original.copy()
np.random.shuffle(shuffled)

* 이후 위에서 구현한 정렬 함수를 사용한다.

In [0]:
sorted = shuffled.copy()
bubblesort(sorted)
print(np.array_equal(sorted, original))

* 이후 수행시간을 확인한다.

In [0]:
sorted[:] = shuffled[:]
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

In [0]:
%timeit sorted[:] = shuffled[:]

### numba.jit을 활용한 함수의 컴파일



* numba를 활용하는 방법을 소개한다.
* numba.jit 데코레이터를 명시적으로 활용하는 방법이 있다.
* 이를 활용한 후 속도 비교를 수행한다.

In [0]:
print(numba.jit.__doc__)

In [0]:
bubblesort_jit = numba.jit("void(f4[:])")(bubblesort)

* 위 코드를 수행하면, bubblesort_jit는 위에서 선언횐 bubblesort를 컴파일한 함수가 된다.
* "void(f4[:])"는 Signature라 불리며 함수의 입력과 리턴을 의미한다.
* f4[:]는 입력, void는 리턴을 의미한다.

* 작동하는지를 확인한다.

In [0]:
sorted[:] = shuffled[:] # reset to shuffled before sorting
bubblesort_jit(sorted)
print(np.array_equal(sorted, original))

* 이제 작동시간을 확인한다.

In [0]:
%timeit sorted[:] = shuffled[:]; bubblesort_jit(sorted)

In [0]:
%timeit sorted[:] = shuffled[:]; bubblesort(sorted)

* 아래와 같은 방식으로도 함수를 컴파일할 수 있다.

In [0]:
@numba.jit("void(f4[:])")
def bubblesort_jit_function(X):
    N = len(X)
    for end in range(N, 1, -1):
        for i in range(end - 1):
            cur = X[i]
            if cur > X[i + 1]:
                tmp = X[i]
                X[i] = X[i + 1]
                X[i + 1] = tmp

In [0]:
sorted[:] = shuffled[:] # reset to shuffled before sorting
bubblesort_jit_function(sorted)
print(np.array_equal(sorted, original))

### Signature
* Python함수를 컴파일할 때 컴파일러는 함수의 입출력 자료형 정보를 받아야만 한다.
* numba.jit을 통해 컴파일된 함수는 알맞은 입력 자료형에 대해서 정상작동한다. 
 * 그렇지 않은 경우도 가능하나, 흔하지는 않다.
* Signature에는 출력 자료형도 포함되며, 아래와 같은 형태를 가진다.
* <출력 자료형>(<첫 번째 입력 자료형>, <두 번째 입력 자료형>, ...)

### Signature가 제공되지 않은 함수의 컴파일
* numba 버전 0.12 이상에서는 numba.jit에 자료형에 대한 Signature가 제공되지 않아도 컴파일이 가능하다.
* 기존에는 이를 numba.autojit로 제공했으나 최근 버전에서는 numba.jit에 포함되었다.

In [0]:
bubblesort_autojit = numba.jit(bubblesort)

In [0]:
%timeit sorted[:] = shuffled[:]; bubblesort_autojit(sorted)

### 몇가지 추가 사항
* 컴파일에는 시간이 필요하다. 작은 함수를 사용하는 경우에 많은 시간이 걸리지는 않을 것이다. 그러나 많은 기능이 있는 함수를 컴파일 할 때는 시간이 오래 걸릴 수 있다. numba는 가능한 한 컴파일을 빠르게 하기 위해 사용하지 않는 함수는 컴파일을 하지 않는다.
* numba를 활용하는 컴파일보다 Python 네이티브 시스템이 더 빠른 경우가 있다. 이 경우에는 컴파일이 되지 않는다.
* 기본적으로, 'cpu' 타겟은 'nopython' 모드로 컴파일한다. 이것이 실패하면 Python 네이티브 시스템으로 사용한다.

In [0]:
@numba.jit("void(i1[:])")
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

from decimal import Decimal
@numba.jit("void(i1[:])")
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

res = np.zeros((10000,), dtype="i1")

In [0]:
%timeit test(res)

In [0]:
%timeit test2(res)

* 'nopython'모드의 코드 컴파일이 실패하는 경우 피드백을 받는 방법은 아래와 같다.
* nopython=True를 통해 강제적으로 nopython모드의 컴파일을 수행하게 함으로써 에러를 받아볼 수 있다.

* 아래 코드는 정상적으로 numba.jit의 'nopython'모드 컴파일이 수행된다.

In [0]:
@numba.jit("void(i1[:])", nopython=True)
def test(value):
    for i in range(len(value)):
        value[i] = i % 100

* 반면에, 아래 코드는 Decimal사용으로 인해 'nopython'모드 컴파일 수행이 불가능하다.

In [0]:
@numba.jit("void(i1[:])", nopython=True)
def test2(value):
    for i in range(len(value)):
        value[i] = i % Decimal(100)

# 과제
## 4번
* 파일 - 새 Python3 노트를 통해 새 노트북을 만드시오.
* Python을 활용하여 구구단을 출력하는 함수를 만드시오.
 * 입력은 하나의 정수를 받으며, 출력은 해당 정수의 구구단을 총 9줄로 출력함.
 * Python 기본 문법을 사용한 방법, numpy를 사용한 방법, numba.jit를 사용한 방법 셋을 전부 활용하여 결과를 비교하시오.
* 파일 - Github에 사본 저장을 통해 1번 과제에서 복사한 저장소에 아래와 같은 이름으로 저장하시오.
 * 열유체공학실험-2016201278-HW04
 * 다른 제목인 경우 미제출 처리 될 수 있음.